In [209]:
from Error_Compare import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [218]:
activity_dir = "data/fa99ac972245361cf4256225cae4a2c0b80ff333"
activity = os.path.split(activity_dir)[-1]
clean_dfs = load_data(activity_dir, has_uncleaned = False)

min_time = 60
max_time = np.inf


# Load Data
raw_chest = clean_dfs["raw_slow_df"][["time","c",]]
key_metrics = ["breathTime", "VE"]#, "VT", "RRAvg", "instBR"]
live_b3_df = clean_dfs["live_b3_df"][key_metrics]
pp_b3_df = clean_dfs["aws_b3_df"][key_metrics]

# Only look at a subset of the data
raw_chest = raw_chest[(raw_chest["time"] > min_time) & (raw_chest["time"] < max_time)]
live_b3_df = live_b3_df[(live_b3_df["breathTime"] > min_time) & (live_b3_df["breathTime"] < max_time)]
pp_b3_df = pp_b3_df[(pp_b3_df["breathTime"] > min_time) & (pp_b3_df["breathTime"] < max_time)]

#live_df = b3_to_seconds(clean_dfs["live_b3_df"][key_metrics])
#pp_df, pp_cleaning_demo = b3_to_seconds(clean_dfs["aws_b3_df"][key_metrics], demo = True)


Pickled data found in activity data directory. Loading data...


# Data Cleaning

In [211]:
# Run the cleaning process
pp_df, pp_cleaning_demo = run_cleaning_process(pp_b3_df[key_metrics], demo = True)

In [212]:
# Plot the result
metric = "VE"
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02)
for demo_name, df in pp_cleaning_demo.items():
    # if demo_name == "og_df":
    #     x = df["breathTime"]
    # else:
    #     x = df.index
    fig.add_trace(go.Scatter(x=df.index, y=df[metric], name = demo_name))

fig.update_layout(title = f"Data Cleaning Demonstration for PP - Metric: {metric}")

fig.show(renderer = "jupyterlab")

## Comparison of PP and Live

In [213]:
live_df = run_cleaning_process(live_b3_df[key_metrics], demo = False)
fig = plot_metrics_compare(raw_chest, live_df, pp_df)
fig.show(renderer = "jupyterlab")

In [219]:
# Settings
error_type = "Percent"
lower_threshold = 0.25
upper_threshold = np.inf

# error_type = "Absolute" #
# lower_threshold = 20
# upper_threshold = np.inf

live_shift = 16


In [215]:

all_error_signals, all_time_in_error, all_error_indices, all_error_fraction, all_error_durations, live_df_sh = perform_error_analysis(live_df, pp_df, key_metrics, live_shift = live_shift, error_type=error_type, lower_threshold=lower_threshold, upper_threshold=upper_threshold)


In [216]:
# Summary
print("Activity: ", activity)
print("Shift: ", live_shift)
print("Error Type: ", error_type)
print("Lower Threshold: ", lower_threshold)
print("Upper Threshold: ", upper_threshold)
print("-"*20)
print("Error Fraction: ", all_error_fraction)

Activity:  fa99ac972245361cf4256225cae4a2c0b80ff333
Shift:  16
Error Type:  Absolute
Lower Threshold:  20
Upper Threshold:  inf
--------------------
Error Fraction:  {'VE': 0.016030956329463792}


In [217]:
# Visualiz Error
fig = plot_metrics_compare(raw_chest, live_df_sh, pp_df, errors_indices=all_error_indices, show = False)
fig.show(renderer = "jupyterlab")